In [1]:
from Scripts import loading as dl, profiling as pf, cleaning as cl


input_link = '../Data/reviews_Movies_and_TV_5.json.gz'
df = dl.load_amazon_full(input_link)

print('######## FIRST GENERAL INSIGHT')
print(df.head())
pf.get_review_count(df)
pf.get_descr(df)


print('######## DATA COMPLETENESS')
pf.get_missing_text(df)
pf.get_missing_label(df)
df = cl.proceed_data_completion(df)


print('######## DUPLICATE DETECTION')
pf.get_duplicates(df)
df = cl.drop_duplicates(df)


# LINGUISTIC AFFILIATION
print('######## LINGUISTIC AFFILIATION')
df = cl.remove_non_english(df)
pf.show_lang_dist(df, 'amazon_movie_lang_non_eng_dist', 'non-English Language Distribution (Amazon Movies & TV)', 0)
pf.show_lang_dist(df, 'amazon_movie_lang_all_dist', 'Language Distribution (Amazon Movies & TV)', 1)


# OTHER HEURISTICS: based on exceptions
print('######## OTHER HEURISTICS')
df = df[~df.text.str.contains(r'^&#((15|16|20)[0-9]{2,3});*')]
print('After removing from other cleaning heuristics: ', len(df))


# STORING VIA PICKLING
print('######## STORING')
# Finally keeping only the wanted properties
df = df[['text', 'label']]
df.to_pickle('../Data/amazon_movie.pkl')
print('to pickle done')



                                                text  label
0  This is a charming version of the classic Dick...    4.0
1  It was good but not as emotionally moving as t...    3.0
2  Don't get me wrong, Winkler is a wonderful cha...    3.0
3  Henry Winkler is very good in this twist on th...    5.0
4  This is one of the best Scrooge movies out.  H...    4.0
######## Total: 
Amount of reviews:  1697533
######## DESCRIPTION
           text         label
count   1697533  1.697533e+06
unique  1696352           NaN
top                       NaN
freq         62           NaN
mean        NaN  4.110648e+00
std         NaN  1.197615e+00
min         NaN  1.000000e+00
25%         NaN  4.000000e+00
50%         NaN  5.000000e+00
75%         NaN  5.000000e+00
max         NaN  5.000000e+00
######## DATA COMPLETENESS
Missing review data: 62
Missing review data as percentage: 0.00365% 
Missing rating information 0
Missing rating information as percentage: 0.00%
After removing missing text and ratings: 

c:\users\d064966\appdata\local\continuum\anaconda3\envs\masterthesis\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
